In [12]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2Model
from tqdm import trange, tqdm

In [13]:
torch.cuda.empty_cache()
torch.cuda.is_available()


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [3]:
data = pd.read_csv('../Dataset/0OYkPK', sep=",", header=None)
data.columns = data.iloc[0]
data = data[1:]
data['full'] = data['sentence1']+ " " + data['sentence2']+ " " + data['sentence3']+ " " + data['sentence4']+ " " + data['sentence5']
data['input'] = data['sentence1']+ " " + data['sentence2']+ " " + data['sentence3']+ " " + data['sentence4']


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model = GPT2Model.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)


In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('device', device)
a = torch.Tensor(5,3)
dp_m = torch.nn.DataParallel(a)
dp_m.to(device)
print(dp_m)

device cuda:1
DataParallel()


In [6]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
#outputs

In [7]:
device = torch.device("cuda:1")
model.to(device)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP

In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#device_map = {0: [0, 1, 2],
#            1: [3, 4,5],
#            2: [6, 7, 8],
#            3: [9,10,11]}
#model = torch.nn.DataParallel(model) # Splits the model across several devices
model.to(device)

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
inputs.to(device)
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)
output = tokenizer.decode(generation_output['sequences'][0])
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, my dog is cute and icky. I'm not sure if she's a good dog


In [11]:
#for i in range(100):
#    #tokenizer.decode(generation_output[0])
#
#    inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
#    inputs.to(device)
#    generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)
#    output = tokenizer.decode(generation_output['sequences'][0])
#    print(output)

In [10]:
for i in tqdm(range(100)):
    # Story is:
    input_ids = tokenizer(data.input.iloc[i], return_tensors='pt')
    input_ids.to(device)
    
    greedy_output = model.generate(**input_ids, return_dict_in_generate=True, output_scores=True, max_length=100)
    output = tokenizer.decode(greedy_output['sequences'][0])
    len_input = len(data.input.iloc[i])
    #output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    pred_sent = output[len_input:output.index('.',len_input)+1]
    expected = data.sentence5.iloc[i]
    
    print('case',  i ,':\n' + 100 * '-')
    print('input:')
    print(data.input.iloc[i])
    print("Expected:")
    print(expected)
    print("Output:")
    print(pred_sent)
    print()

  1%|          | 1/100 [00:00<01:15,  1.31it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


case 0 :
----------------------------------------------------------------------------------------------------
input:
David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet.
Expected:
After a few weeks, he started to feel much better.
Output:


"I'm not a big fan of fast food," he said.



  2%|▏         | 2/100 [00:01<01:24,  1.16it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


case 1 :
----------------------------------------------------------------------------------------------------
input:
Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly.
Expected:
Tom sat on his couch filled with regret about his actions.
Output:


The next day, Tom was in the hospital.



  3%|▎         | 3/100 [00:02<01:16,  1.27it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

case 75 :
----------------------------------------------------------------------------------------------------
input:
The candidates were running a tight campaign. The latest poll said they were tied. The Republican candidate focused on radio ads. The Democrat focused on TV.
Expected:
They fought for every vote they could grab.
Output:


The poll was conducted by telephone from Oct.



 77%|███████▋  | 77/100 [00:52<00:15,  1.44it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


case 76 :
----------------------------------------------------------------------------------------------------
input:
Johnnie got dragged to the ballet because his sister wanted to go. He was dreading how boring it would be. He was already bored when the lights were dimmed. Then when the dancers started he got into it.
Expected:
He was sad when it ended.
Output:
 He was so excited.



 77%|███████▋  | 77/100 [00:52<00:15,  1.46it/s]


ValueError: substring not found

In [15]:
#tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
#model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

#device = torch.device('cuda:0')
#model.to(device)

#model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

#device_map = {0: [0, 1, 2],
#            1: [3, 4,5],
#            2: [6, 7, 8],
#            3: [9,10,11]}
#model.parallelize(device_map) # Splits the model across several devices


# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')
# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))



ImportError: Unable to convert output to TensorFlow tensors format, TensorFlow is not installed.

In [ ]:
for i in tqdm(range(100)):
    # Story is:
    input_ids = tokenizer.encode(data.input.iloc[i], return_tensors='tf')
    
    greedy_output = model.generate(input_ids, max_length=100)
    len_input = len(data.input.iloc[i])
    output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    pred_sent = output[len_input:output.index('.',len_input)+1]
    expected = data.sentence5.iloc[i]
    
    print('case',  i ,':\n' + 100 * '-')
    print('input:')
    print(data.input.iloc[i])
    print("Expected:")
    print(expected)
    print("Output:")
    print(pred_sent)
    print()
